<a href="https://colab.research.google.com/github/IrakizaGaius/activity-recognition-hmm/blob/main/Hmm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary Dependencies

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 4.4 MB/s eta 0:00:00


# Importing All the neccessary Dependencies

#Load and unzip activity data files

In [7]:
from google.colab import files
uploaded = files.upload()


Saving Activity_Data-20251028T081807Z-1-001.zip to Activity_Data-20251028T081807Z-1-001.zip


In [13]:
!unzip Activity_Data.zip

Archive:  Activity_Data.zip
  inflating: Activity_Data/Standing/Standing_Gyroscope_combined.csv  
  inflating: Activity_Data/Walking/Walking_Gyroscope_combined.csv  
  inflating: Activity_Data/Walking/Walking_Accelerometer_combined.csv  
  inflating: Activity_Data/Still/Still_Gyroscope_combined.csv  
  inflating: Activity_Data/Still/Still_Accelerometer_combined.csv  
  inflating: Activity_Data/Jumping/Jumping_Accelerometer_combined.csv  
  inflating: Activity_Data/Jumping/Jumping_Gyroscope_combined.csv  
  inflating: Activity_Data/Standing/Standing_Accelerometer_combined.csv  


# Feature Extraction Configs

In [14]:
base_dir = 'Activity_Data'
activities = ['Jumping', 'Standing', 'Still', 'Walking']

TARGET_RATE = 50
WINDOW_SIZE = 150
STEP_SIZE = 50
SAMPLE_PERIOD = 1 / TARGET_RATE

# Feature Extraction & Data Sampling Functions

In [15]:
def estimate_sampling_rate(df):
    if 'seconds_elapsed' not in df.columns:
        raise ValueError("Missing 'seconds_elapsed' column in dataset.")
    diffs = np.diff(df['seconds_elapsed'])
    avg_period = np.mean(diffs)
    return 1.0 / avg_period if avg_period > 0 else TARGET_RATE

def resample_signal(df, target_rate=50):
    if 'seconds_elapsed' not in df.columns:
        raise ValueError("Missing 'seconds_elapsed' column for resampling.")

    t = df['seconds_elapsed'].values
    start, end = t.min(), t.max()
    target_period = 1.0 / target_rate
    new_time = np.arange(start, end, target_period)

    df_interp = pd.DataFrame({'seconds_elapsed': new_time})
    for axis in ['x', 'y', 'z']:
        df_interp[axis] = np.interp(new_time, t, df[axis])
    return df_interp

def extract_features(df, sensor_type, window_size=WINDOW_SIZE, step_size=STEP_SIZE):
    features = []
    num_windows = (len(df) - window_size) // step_size + 1
    for w in range(num_windows):
        start = w * step_size
        window = df.iloc[start:start + window_size]
        row = {}

        acc_x = window['x'].values
        acc_y = window['y'].values
        acc_z = window['z'].values

        for axis, signal in zip(['x','y','z'], [acc_x, acc_y, acc_z]):
            row[f'{sensor_type}_{axis}_rms'] = np.sqrt(np.mean(signal ** 2))
            row[f'{sensor_type}_{axis}_var'] = np.var(signal)
            row[f'{sensor_type}_{axis}_mean_abs'] = np.mean(np.abs(signal))

            fft_vals = np.abs(rfft(signal))
            fft_freqs = rfftfreq(window_size, d=SAMPLE_PERIOD)
            row[f'{sensor_type}_{axis}_dom_freq'] = fft_freqs[np.argmax(fft_vals)]

            row[f'{sensor_type}_{axis}_fft_energy'] = np.sum(fft_vals**2)
            low_freq_mask = fft_freqs < 5
            row[f'{sensor_type}_{axis}_lowfreq_energy'] = np.sum(fft_vals[low_freq_mask]**2)
            row[f'{sensor_type}_{axis}_fft_var'] = np.var(fft_vals)

        smv = np.sqrt(acc_x**2 + acc_y**2 + acc_z**2)
        row[f'{sensor_type}_smv_rms'] = np.sqrt(np.mean(smv**2))
        row[f'{sensor_type}_smv_var'] = np.var(smv)
        row[f'{sensor_type}_smv_mean_abs'] = np.mean(np.abs(smv))

        features.append(row)

    return pd.DataFrame(features)

# Call Feature extraction & Data sampling Functions

# HMM Model structure and Training